In [1]:
import sys, os
print (sys.version)
import cv2, math
import time
from matplotlib import pyplot as plt
import signal
import numpy as np
import multiprocessing
import threading
from threading import RLock


print(cv2.__file__)
%matplotlib notebook

3.6.8 |Anaconda, Inc.| (default, Dec 30 2018, 01:22:34) 
[GCC 7.3.0]
/home/yousof/Software/programming/anaconda3/envs/python36/lib/python3.6/site-packages/cv2.cpython-36m-x86_64-linux-gnu.so


# Import and Setup OpenPose

In [2]:
openPoseDir = "/home/yousof/AI/openpose/"
openPoseLib = openPoseDir + "build/python"
sys.path.append(openPoseLib);
from openpose import pyopenpose as op
import openpose

params = dict()
params["model_folder"] = openPoseDir + "models"
params["hand"] = True
params["hand_detector"] = 0

In [3]:
def plotOpenCVImage(img):
    plt.figure(figsize=(10,18))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

# Initialize OpenPose

In [4]:
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

In [5]:
def pointsToDegrees(d):
    BaseIdx = 5
    midIdx = 6
    tipIdx = 8
    hand = 0
    if d.handKeypoints[hand].size != 1:
        points = d.handKeypoints[hand][0, :, :].squeeze()
        all_zeros = not np.any(points)
        if not all_zeros:
            points[:, 1] = d.cvOutputData.shape[1] - points[:, 1]
            dy = points[midIdx, 1] - points[BaseIdx, 1]
            dx = points[midIdx, 0] - points[BaseIdx, 0]
            deg1 = math.degrees(math.atan2(dy, dx))
            dy = points[tipIdx, 1] - points[midIdx, 1]
            dx = points[tipIdx, 0] - points[midIdx, 0]
            deg2 = math.degrees(math.atan2(dy, dx))
            print(deg1 - 90, deg2)
    
def processImage(img):
    global  opWrapper
    datum = op.Datum()
    datum.cvInputData = img
    opWrapper.emplaceAndPop([datum])
    pointsToDegrees(datum)

# Read from webcam

In [6]:
bufferLock = RLock()
image = None
isNewImage = False
running = True
def readImages():
    global running, image, isNewImage, bufferLock, cam
    cam = cv2.VideoCapture(0)
    cam.set(3 , 640  ) # width         1920
    cam.set(4 , 480  ) # height        1080      
    print("cam.isOpened()", cam.isOpened())
    while cam.isOpened() and running:
        ret, img = cam.read()
        if ret:
            bufferLock.acquire()
            isNewImage = True
            image = img.copy()
            bufferLock.release()
        else:
            break
    print("Releasing Camera!")
    cam.release()

imageReaderThread = threading.Thread(target=readImages)

In [7]:
def signal_handler(signal, frame):
    global running
    running = False
    
signal.signal(signal.SIGINT, signal_handler)

<function _signal.default_int_handler>

In [8]:
imageReaderThread.start()

In [9]:
def captureLastFrame():
    global image, isNewImage, bufferLock
    lastFrame = None
    suc = False
    if isNewImage:
        bufferLock.acquire()
        isNewImage = False
        lastFrame = image.copy()
        bufferLock.release()
        suc = True
    #print("captureLastFrame:", suc)
    return suc, lastFrame

In [10]:
while running:
    ret, frame = captureLastFrame()
    if ret:
        processImage(frame)

cam.isOpened() True
-9.46231309289486 42.165896107505084
-16.504373213330254 43.19130498980996
-19.093541521301162 39.58813299556067
-22.782399086988107 40.95551971542824
-24.274486789536738 42.662727901136115
-19.983087307005448 40.3331293757782
-4.484623109758729 64.23065233964562
-2.2457633612377492 62.62231325369587
-2.2025911635624027 63.43499394465494
-3.2396935104653437 62.15242440120889
-7.989366556627502 63.43494882292201
-8.13007816348248 61.03233541761327
-2.1210854209988526 62.281517130727785
-6.115519681100963 64.57312933190475
-7.386020774447644 58.448617876351825
-10.684896110192085 49.18494603409396
-12.407393301537482 41.00909876324564
-18.083438209618294 29.0546090937852
-21.801360801302266 21.97442895225422
-22.01126836509296 14.50019774572938
-20.556081612552006 10.222169463419068
-22.442766237866095 -2.045432859890352
-20.95580441990667 -8.13011829436647
-19.88516766684556 -9.293303466288354
-21.37063272319311 -11.113028788601595
-21.37059874894365 -16.504345299428

KeyboardInterrupt: 

running=False